In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device('mps')

In [10]:
def make_batch():
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()  # space tokenizer
        input = [word_dict[n] for n in word[:-1]]  # create (1~n-1) as input
        target = word_dict[word[-1]]  # create (n) as target, We usually call this 'casual language model'

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [17]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden).to(device)
        self.W = nn.Linear(n_hidden, n_class, bias=False).to(device)
        self.b = nn.Parameter(torch.ones([n_class])).to(device)

    def forward(self, hidden, X): # X [3, 2, 7], hidden [1, 3, 5]
        X = X.transpose(0, 1).to(device) # X [2, 3, 7]
        outputs, hidden = self.rnn(X, hidden) # outputs [2, 3, 5], hidden [1, 3, 5]
        outputs = outputs[-1] # outputs [3, 5]
        model = self.W(outputs) + self.b # model [3, 7]
        return model

In [24]:
if __name__ == '__main__':
    n_step = 2
    n_hidden = 5

    sentences = ["i like dog", "i love coffee", "i hate milk"]

    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    number_dict = {i: w for i, w in enumerate(word_list)}
    n_class = len(word_dict)
    batch_size = len(sentences)

    model = TextRNN()

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    input_batch, target_bath = make_batch()
    input_batch = torch.FloatTensor(input_batch).to(device)
    target_batch = torch.LongTensor(target_bath).to(device)

    # Training
    for epoch in range(5000):
        optimizer.zero_grad()

        hidden = torch.zeros(1, batch_size, n_hidden).to(device)
        output = model(hidden, input_batch) # output : [3, 7]

        loss = criterion(output, target_batch).to(device) # target_batch : [3]
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        
        loss.backward()
        optimizer.step()
    
    input = [sen.split()[:2] for sen in sentences]

    # Predict
    hidden = torch.zeros(1, batch_size, n_hidden).to(device)
    predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])
        

Epoch: 1000 cost = 0.084289
Epoch: 2000 cost = 0.019321
Epoch: 3000 cost = 0.007951
Epoch: 4000 cost = 0.003965
Epoch: 5000 cost = 0.002160
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
